# Prevalence of PRIMIS codelists

In [1]:
import pandas as pd
import numpy as np
import os

suffix = f"_{os.environ.get('OPENSAFELY_BACKEND', 'tpp')}"
os.makedirs(os.path.join("..","safe-outputs"), exist_ok=True)

### Load data

In [2]:
df = pd.read_csv(os.path.join("..","output","input.csv"))

if "hashed_organisation" in df.columns:
    df = df.drop("hashed_organisation", axis=1)

for col in df.columns:
    if col in ["patient_id", "age", "sex"]:
        continue
    # Most columns only contain years or NaN so we can store them as
    # float16s, which saves a lot of memory
    df[col] = df[col].astype("float16")

### Create ageband

In [3]:
agebands = ['16-39', '40-69', '70+']
conditions = [
    (df['age'] >= 16) & (df['age'] < 40),
    (df['age'] >= 40) & (df['age'] < 70),
    (df['age'] >= 70) & (df['age'] < 120)]
choices = agebands
df['ageband'] = np.select(conditions, choices, default=np.nan)

# filter to largest sex groups
df['sex'] = df['sex'].replace(['I','U'], np.nan)

### Summarise data

In [4]:
# list columns of interest 
cols_allyears = [c for c in df.columns if c not in ["age","patient_id"]]
cols_recent = ["preg", "pregdel"]

In [5]:
# filter to valid sexes and agegroups only
df1 = df.copy().loc[(df["sex"].isin(["F","M"])) & (df["ageband"].isin(agebands))]

### Calculate population denominators

In [6]:

out2 = df1.groupby(["ageband", "sex"])[["patient_id"]].nunique().rename(columns={"patient_id":"total_population"}).transpose()

# calculate total population across all ages and sexes
out2["total"] = out2.sum(axis=1)

out2

ageband          16-39       40-69        70+      total
sex                  F     M     F     M    F    M      
total_population  1533  1543  1773  1862  657  630  7998

### Codelist counts

In [7]:
# for codes that are only relevant if recent (pregnancy/delivery), remove any older dates
for c in cols_recent:
    df1.loc[(df1[c]<2020), c] = np.nan

# summarise by age and gender
out = df1[cols_allyears].groupby(["ageband", "sex"]).count().transpose()

# suppress low numbers
out = out.replace([0,1,2,3,4],0)

# calculate total count for each codelist across all ages and sexes
out["total"] = out.sum(axis=1)

# add population denominators
out = pd.concat([out,out2])

out.tail()

ageband          16-39       40-69        70+      total
sex                  F     M     F     M    F    M      
shield             152   149   200   177   61   66   805
smhres             169   160   169   191   62   57   808
spln_cov           181   171   178   158   58   60   806
registered        1533  1543  1773  1862  657  630  7998
total_population  1533  1543  1773  1862  657  630  7998

### Codelist prevalence rates

In [8]:
# calculate rates
for i in out.index.drop("total_population"):
    out.loc[i] = (1000*out.loc[i]/out.loc["total_population"]).round(1)

# export to csv    
out.to_csv(os.path.join("..","safe-outputs",f"code-prevalence-by-age-and-sex{suffix}.csv"))

out

ageband            16-39           40-69             70+           total
sex                    F       M       F       M       F       M        
ast                 98.5   110.2    97.0   106.3    85.2   109.5   102.0
astadm             104.4   108.9    84.0   104.7   117.2    92.1   100.9
astrx              110.2   103.7    92.5   105.3    82.2   104.8   101.2
bmi                 95.2   106.9   108.9    98.3   103.5    96.8   102.0
bmi_stage           99.2    99.8    97.6    99.4    91.3   119.0    99.9
carehome            98.5   101.1   112.2    97.7    86.8    93.7   100.5
carer              104.4    99.2   105.5    96.1    95.9   112.7   101.7
chd_cov             90.7    98.5   107.7    91.3    95.9    95.2    96.9
ckd15              105.0    95.9   109.4    96.1   118.7    84.1   101.7
ckd35               92.0   109.5   100.4    94.0    88.3   125.4   100.0
ckd_cov             99.8   103.7    99.8    98.8   109.6    84.1    99.9
cld                 98.5   103.0   112.2    97.2   100.5    93.7   101.9
cns_cov             92.0    95.3   103.2    94.5   121.8   114.3    99.9
cov1decl           100.5    95.3   102.1   103.7    80.7   115.9   100.2
cov2decl           103.7    95.9   103.8   102.6   109.6    85.7   101.0
covadm1            105.7   112.1   108.3    91.8    86.8    84.1   101.0
covadm2             95.2    92.7   102.1    95.1   127.9   109.5   100.0
covcontra           95.2    95.9   110.0   103.1   100.5   104.8   101.7
covrx               95.9   104.3    99.8   105.8    97.4   101.6   101.3
diab               106.3    89.4    95.3   105.8   105.0   111.1   100.8
dmres               93.9    94.6    97.6   113.9    92.8    82.5    98.5
domcare             90.7    99.2    98.1    97.2   100.5   109.5    97.8
eth2001             99.2    95.3    95.3   101.0   106.5    85.7    97.5
eth_norecord        89.4   104.3   104.3    84.9   133.9    96.8    98.8
eth_notgiptref     110.9   101.7    98.7   102.0    94.4    84.1   100.9
eth_notstated      103.1   108.9    93.1   107.9    82.2    90.5   100.4
hhld_imdef         110.2    98.5    83.5    94.0    91.3   109.5    96.6
immdx_cov          110.2   108.2    91.9   104.2    92.8    84.1   100.9
immrx              105.0    96.6   100.4    98.8   121.8   104.8   102.3
learndis           101.8    85.5    91.9   102.0   112.6   117.5    98.6
longres            105.7    95.9    99.3   109.6    86.8   103.2   101.5
non_eth2001         92.6   106.3   104.3    95.6    91.3   112.7   100.0
nonshield          101.8    90.7    89.1   109.6   114.2    93.7    99.0
notcarer           100.5   110.8   107.7    94.5    94.4    79.4   100.5
nursehome           87.4   100.5   102.7   104.2    95.9    93.7    98.4
preg                 0.0     5.2     0.0     3.2     0.0     0.0     1.8
pregdel              3.9     3.2     0.0     0.0     0.0     9.5     2.1
resp_cov           109.6    97.9   104.3    89.7    89.8    98.4    99.0
sev_mental          86.1   102.4   102.1    95.6   123.3    87.3    98.1
sev_obesity         90.0    86.2   103.2   100.4   115.7   123.8    99.4
shield              99.2    96.6   112.8    95.1    92.8   104.8   100.7
smhres             110.2   103.7    95.3   102.6    94.4    90.5   101.0
spln_cov           118.1   110.8   100.4    84.9    88.3    95.2   100.8
registered        1000.0  1000.0  1000.0  1000.0  1000.0  1000.0  1000.0
total_population  1533.0  1543.0  1773.0  1862.0   657.0   630.0  7998.0